<a href="https://colab.research.google.com/github/pratikiiitb2013/EVA4/blob/master/Session4/EVA4S4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3, padding=1, bias=False) #input -? OUtput? RF
        self.conv2 = nn.Conv2d(16, 20, 3, padding=1, bias=False)
        self.conv3 = nn.Conv2d(20, 30, 3, padding=1, bias=False)
        self.conv4 = nn.Conv2d(20, 16, 1, bias=False)
        self.conv5 = nn.Conv2d(30, 16, 1, bias=False)
        self.conv6 = nn.Conv2d(20, 16, 3, bias=False)
        self.conv7 = nn.Conv2d(16, 10, 3, bias=False)
        self.adpt  = nn.AdaptiveAvgPool2d((1,1))
        self.pool1 = nn.MaxPool2d(2, 2)
        self.bn_16_1 = nn.BatchNorm2d(16)
        self.bn_16_2 = nn.BatchNorm2d(16)
        self.bn_16_3 = nn.BatchNorm2d(16)
        self.bn_24_1 = nn.BatchNorm2d(20)
        self.bn_24_2 = nn.BatchNorm2d(20)
        self.bn_24_3 = nn.BatchNorm2d(20)
        self.bn_32_1 = nn.BatchNorm2d(30)
        self.bn_32_2 = nn.BatchNorm2d(30)
        self.bn_32_3 = nn.BatchNorm2d(30)
        self.dout = nn.Dropout(0.10)

    def forward(self, x):
      x = self.bn_16_1(F.relu(self.conv1(x)))
      x = self.bn_24_1(F.relu(self.conv2(x)))
      # x = self.dout(self.bn_24_1(F.relu(self.conv3(x))))
      x = self.pool1(x)
      x = self.dout(self.bn_16_2(F.relu(self.conv4(x))))
      x = self.dout(self.bn_24_2(F.relu(self.conv2(x))))
      x = self.dout(self.bn_32_2(F.relu(self.conv3(x))))
      x = self.pool1(x)
      x = self.dout(self.bn_16_3(F.relu(self.conv5(x))))
      x = self.dout(self.bn_24_3(F.relu(self.conv2(x))))
      # x = self.dout(self.bn_32_3(F.relu(self.conv3(x))))
      # x = self.conv4(x)
      x = self.adpt(self.conv7(self.conv6(x)))
      x = x.view(-1, 10)
      return F.log_softmax(x)




In [ ]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 28, 28]             144
       BatchNorm2d-2           [-1, 16, 28, 28]              32
            Conv2d-3           [-1, 20, 28, 28]           2,880
       BatchNorm2d-4           [-1, 20, 28, 28]              40
         MaxPool2d-5           [-1, 20, 14, 14]               0
            Conv2d-6           [-1, 16, 14, 14]             320
       BatchNorm2d-7           [-1, 16, 14, 14]              32
           Dropout-8           [-1, 16, 14, 14]               0
            Conv2d-9           [-1, 20, 14, 14]           2,880
      BatchNorm2d-10           [-1, 20, 14, 14]              40
          Dropout-11           [-1, 20, 14, 14]               0
           Conv2d-12           [-1, 30, 14, 14]           5,400
      BatchNorm2d-13           [-1, 30, 14, 14]              60
          Dropout-14           [-1, 30,

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:39: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [ ]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [ ]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    accuracy=0
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)

        ps = torch.exp(output)                
        top_p, top_class = ps.topk(1, dim=1)
        equals = top_class == target.view(*top_class.shape)
        accuracy += torch.mean(equals.type(torch.FloatTensor))

        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        # pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')
        pbar.set_description(desc= f'accuracy={accuracy/len(train_loader)} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [ ]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 21):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0460, Accuracy: 9858/10000 (99%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0503, Accuracy: 9835/10000 (98%)



  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0216, Accuracy: 9930/10000 (99%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0181, Accuracy: 9941/10000 (99%)



accuracy=0.9932702779769897 batch_id=468: 100%|██████████| 469/469 [00:13<00:00, 34.88it/s]



Test set: Average loss: 0.0166, Accuracy: 9953/10000 (100%)

